In [6]:
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

https://deeplearningsandbox.com/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2

In [8]:
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172 # unfreeze the top 2 inception blocks

def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    
    for layer in model.layers:
        print(layer.name, ': ', layer.trainable)
    model.compile(optimizer=RMSprop(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])


def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
      Args:
        base_model: keras model excluding top
        nb_classes: # of classes
      Returns:
        new keras model with last layer
    """
    
    x = base_model.output

    # GlobalAveragePooling2D converts the MxNxC tensor output into a 1xC tensor where C is the # of channels.
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) # new FC layer, random init
    # softmax function on the output to squeeze the values between [0,1]
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
  note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
  Args:
    model: keras model
  """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


def train():
    """Use transfer learning and fine-tuning to train a network on a new dataset"""
    
    nb_train_samples = 45000
    nb_classes = 15
    nb_val_samples = 15000
    nb_epoch = 15
    batch_size = 4 #16
    train_dir = 'data-all/train'
    val_dir = 'data-all/validation'
    output_model_file = 'inceptionv3_clothing_expanded_classifier.h5'

    # data prep
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input, # zero-centers our image data
        # rescale = 1./255,
#           rotation_range=30,
#           width_shift_range=0.2,
#           height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    
    val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input
    )
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # setup model
    # leave out the weights of the last fully connected layer
    base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, nb_classes)
    
    # transfer learning
    setup_to_transfer_learn(model, base_model)

    history_tl = model.fit_generator(
        train_generator,
        epochs=nb_epoch,
        steps_per_epoch=nb_train_samples//batch_size,
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size
        #,
        #class_weight='auto'
    )
    
    print('Begin Fine Tuning')
    # fine-tuning
    setup_to_finetune(model)
    
    history_ft = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size
        #,
        #class_weight='auto'
    )
    
    model_json = model.to_json()
    with open("incep_filter_clothing_expanded_classifier.json", 'w') as json_file:
        json_file.write(model_json)
    
    model.save(output_model_file)


In [9]:
train()

Found 44999 images belonging to 15 classes.
Found 15000 images belonging to 15 classes.
input_4 :  False
conv2d_234 :  False
batch_normalization_233 :  False
activation_233 :  False
conv2d_235 :  False
batch_normalization_234 :  False
activation_234 :  False
conv2d_236 :  False
batch_normalization_235 :  False
activation_235 :  False
max_pooling2d_12 :  False
conv2d_237 :  False
batch_normalization_236 :  False
activation_236 :  False
conv2d_238 :  False
batch_normalization_237 :  False
activation_237 :  False
max_pooling2d_13 :  False
conv2d_242 :  False
batch_normalization_241 :  False
activation_241 :  False
conv2d_240 :  False
conv2d_243 :  False
batch_normalization_239 :  False
batch_normalization_242 :  False
activation_239 :  False
activation_242 :  False
average_pooling2d_23 :  False
conv2d_239 :  False
conv2d_241 :  False
conv2d_244 :  False
conv2d_245 :  False
batch_normalization_238 :  False
batch_normalization_240 :  False
batch_normalization_243 :  False
batch_normalizatio

11250/11250 [==============================] - 2276s - loss: 1.1573 - acc: 0.6384 - val_loss: 0.8446 - val_acc: 0.7325
Epoch 3/15
11250/11250 [==============================] - 2276s - loss: 1.0790 - acc: 0.6586 - val_loss: 0.7997 - val_acc: 0.7408
Epoch 4/15
11250/11250 [==============================] - 2279s - loss: 1.0392 - acc: 0.6727 - val_loss: 0.8058 - val_acc: 0.7425
Epoch 5/15
11250/11250 [==============================] - 2286s - loss: 1.0042 - acc: 0.6814 - val_loss: 0.7682 - val_acc: 0.7517
Epoch 6/15
11250/11250 [==============================] - 2281s - loss: 0.9862 - acc: 0.6871 - val_loss: 0.7574 - val_acc: 0.7566
Epoch 7/15
11250/11250 [==============================] - 2283s - loss: 0.9721 - acc: 0.6901 - val_loss: 0.7462 - val_acc: 0.7597
Epoch 8/15
11250/11250 [==============================] - 2279s - loss: 0.9521 - acc: 0.6955 - val_loss: 0.7543 - val_acc: 0.7605
Epoch 9/15
11250/11250 [==============================] - 2279s - loss: 0.9410 - acc: 0.6989 - val_lo

### Basic Clothing Classifier
- Epoch 1/10
7500/7500 [==============================] - 1521s - loss: 1.6899 - acc: 0.4530 - val_loss: 1.2355 - val_acc: 0.6033
- Epoch 2/10
7500/7500 [==============================] - 1516s - loss: 1.3493 - acc: 0.5525 - val_loss: 1.0849 - val_acc: 0.6403
- Epoch 3/10
7500/7500 [==============================] - 1516s - loss: 1.2620 - acc: 0.5773 - val_loss: 1.0180 - val_acc: 0.6596
- Epoch 4/10
7500/7500 [==============================] - 1516s - loss: 1.2161 - acc: 0.5904 - val_loss: 1.0001 - val_acc: 0.6623
- Epoch 5/10
7500/7500 [==============================] - 1516s - loss: 1.1917 - acc: 0.5996 - val_loss: 0.9620 - val_acc: 0.6761
- Epoch 6/10
7500/7500 [==============================] - 1516s - loss: 1.1749 - acc: 0.6036 - val_loss: 0.9482 - val_acc: 0.6784
- Epoch 7/10
7500/7500 [==============================] - 1516s - loss: 1.1600 - acc: 0.6087 - val_loss: 0.9682 - val_acc: 0.6797
- Epoch 8/10
7500/7500 [==============================] - 1517s - loss: 1.1409 - acc: 0.6126 - val_loss: 0.9424 - val_acc: 0.6864
- Epoch 9/10
7500/7500 [==============================] - 1521s - loss: 1.1387 - acc: 0.6142 - val_loss: 0.9183 - val_acc: 0.6915
- Epoch 10/10
7500/7500 [==============================] - 1518s - loss: 1.1330 - acc: 0.6178 - val_loss: 0.9403 - val_acc: 0.6893
Begin Fine Tuning
- Epoch 1/10
7500/7500 [==============================] - 2327s - loss: 1.0142 - acc: 0.6676 - val_loss: 0.6162 - val_acc: 0.7978
- Epoch 2/10
7500/7500 [==============================] - 2333s - loss: 0.6800 - acc: 0.7705 - val_loss: 0.6005 - val_acc: 0.8130
- Epoch 3/10
7500/7500 [==============================] - 2333s - loss: 0.5676 - acc: 0.8071 - val_loss: 0.5178 - val_acc: 0.8349
- Epoch 4/10
7500/7500 [==============================] - 2333s - loss: 0.5044 - acc: 0.8279 - val_loss: 0.5559 - val_acc: 0.8302
- Epoch 5/10
7500/7500 [==============================] - 2333s - loss: 0.4558 - acc: 0.8435 - val_loss: 0.4924 - val_acc: 0.8420
- Epoch 6/10
7500/7500 [==============================] - 2333s - loss: 0.4254 - acc: 0.8535 - val_loss: 0.4852 - val_acc: 0.8448
- Epoch 7/10
7500/7500 [==============================] - 2333s - loss: 0.3930 - acc: 0.8653 - val_loss: 0.5180 - val_acc: 0.8360
- Epoch 8/10
7500/7500 [==============================] - 2333s - loss: 0.3661 - acc: 0.8741 - val_loss: 0.4862 - val_acc: 0.8535
- Epoch 9/10
7500/7500 [==============================] - 2333s - loss: 0.3428 - acc: 0.8802 - val_loss: 0.4830 - val_acc: 0.8547
- Epoch 10/10
7500/7500 [==============================] - 2334s - loss: 0.3217 - acc: 0.8877 - val_loss: 0.5089 - val_acc: 0.8519

### Expanded Clothing Classifier
Epoch 1/15
11250/11250 [==============================] - 2286s - loss: 1.5958 - acc: 0.5364 - val_loss: 0.9745 - val_acc: 0.6983
Epoch 2/15
11250/11250 [==============================] - 2276s - loss: 1.1573 - acc: 0.6384 - val_loss: 0.8446 - val_acc: 0.7325
Epoch 3/15
11250/11250 [==============================] - 2276s - loss: 1.0790 - acc: 0.6586 - val_loss: 0.7997 - val_acc: 0.7408
Epoch 4/15
11250/11250 [==============================] - 2279s - loss: 1.0392 - acc: 0.6727 - val_loss: 0.8058 - val_acc: 0.7425
Epoch 5/15
11250/11250 [==============================] - 2286s - loss: 1.0042 - acc: 0.6814 - val_loss: 0.7682 - val_acc: 0.7517
Epoch 6/15
11250/11250 [==============================] - 2281s - loss: 0.9862 - acc: 0.6871 - val_loss: 0.7574 - val_acc: 0.7566
Epoch 7/15
11250/11250 [==============================] - 2283s - loss: 0.9721 - acc: 0.6901 - val_loss: 0.7462 - val_acc: 0.7597
Epoch 8/15
11250/11250 [==============================] - 2279s - loss: 0.9521 - acc: 0.6955 - val_loss: 0.7543 - val_acc: 0.7605
Epoch 9/15
11250/11250 [==============================] - 2279s - loss: 0.9410 - acc: 0.6989 - val_loss: 0.7582 - val_acc: 0.7593
Epoch 10/15
11250/11250 [==============================] - 2283s - loss: 0.9374 - acc: 0.6994 - val_loss: 0.7659 - val_acc: 0.7635
Epoch 11/15
11250/11250 [==============================] - 2279s - loss: 0.9220 - acc: 0.7076 - val_loss: 0.7433 - val_acc: 0.7651
Epoch 12/15
11250/11250 [==============================] - 2280s - loss: 0.9109 - acc: 0.7090 - val_loss: 0.7346 - val_acc: 0.7693
Epoch 13/15
11250/11250 [==============================] - 2282s - loss: 0.9066 - acc: 0.7098 - val_loss: 0.7377 - val_acc: 0.7707
Epoch 14/15
11250/11250 [==============================] - 2284s - loss: 0.8922 - acc: 0.7129 - val_loss: 0.7481 - val_acc: 0.7697
Epoch 15/15
11250/11250 [==============================] - 2283s - loss: 0.9039 - acc: 0.7104 - val_loss: 0.7430 - val_acc: 0.7713
Begin Fine Tuning
Epoch 1/15
11250/11250 [==============================] - 3508s - loss: 0.8288 - acc: 0.7496 - val_loss: 0.5294 - val_acc: 0.8499
Epoch 2/15
11250/11250 [==============================] - 3512s - loss: 0.5151 - acc: 0.8325 - val_loss: 0.4399 - val_acc: 0.8710
Epoch 3/15
11250/11250 [==============================] - 3521s - loss: 0.4273 - acc: 0.8605 - val_loss: 0.4955 - val_acc: 0.8644
Epoch 4/15
11250/11250 [==============================] - 3521s - loss: 0.3815 - acc: 0.8736 - val_loss: 0.3975 - val_acc: 0.8875
Epoch 5/15
11250/11250 [==============================] - 3520s - loss: 0.3416 - acc: 0.8856 - val_loss: 0.3986 - val_acc: 0.8890
Epoch 6/15
11250/11250 [==============================] - 3509s - loss: 0.3160 - acc: 0.8938 - val_loss: 0.3929 - val_acc: 0.8903
Epoch 7/15
11250/11250 [==============================] - 3508s - loss: 0.2929 - acc: 0.9014 - val_loss: 0.4139 - val_acc: 0.8883
Epoch 8/15
11250/11250 [==============================] - 3510s - loss: 0.2747 - acc: 0.9077 - val_loss: 0.3906 - val_acc: 0.8901
Epoch 9/15
11250/11250 [==============================] - 3515s - loss: 0.2544 - acc: 0.9137 - val_loss: 0.4232 - val_acc: 0.8867
Epoch 10/15
11250/11250 [==============================] - 3520s - loss: 0.2371 - acc: 0.9192 - val_loss: 0.3987 - val_acc: 0.8927
Epoch 11/15
11250/11250 [==============================] - 3521s - loss: 0.2196 - acc: 0.9229 - val_loss: 0.4391 - val_acc: 0.8813
Epoch 12/15
11250/11250 [==============================] - 3516s - loss: 0.2088 - acc: 0.9292 - val_loss: 0.4348 - val_acc: 0.8832
Epoch 13/15
11250/11250 [==============================] - 3519s - loss: 0.2014 - acc: 0.9308 - val_loss: 0.4215 - val_acc: 0.8880
Epoch 14/15
11250/11250 [==============================] - 3521s - loss: 0.1868 - acc: 0.9341 - val_loss: 0.4340 - val_acc: 0.8855
Epoch 15/15
11250/11250 [==============================] - 3516s - loss: 0.1758 - acc: 0.9393 - val_loss: 0.4352 - val_acc: 0.8895
Basic Clothing Classifier
Epoch 1/10 7500/7500 [==============================] - 1521s - loss: 1.6899 - acc: 0.4530 - val_loss: 1.2355 - val_acc: 0.6033
Epoch 2/10 7500/7500 [==============================] - 1516s - loss: 1.3493 - acc: 0.5525 - val_loss: 1.0849 - val_acc: 0.6403
Epoch 3/10 7500/7500 [==============================] - 1516s - loss: 1.2620 - acc: 0.5773 - val_loss: 1.0180 - val_acc: 0.6596
Epoch 4/10 7500/7500 [==============================] - 1516s - loss: 1.2161 - acc: 0.5904 - val_loss: 1.0001 - val_acc: 0.6623
Epoch 5/10 7500/7500 [==============================] - 1516s - loss: 1.1917 - acc: 0.5996 - val_loss: 0.9620 - val_acc: 0.6761
Epoch 6/10 7500/7500 [==============================] - 1516s - loss: 1.1749 - acc: 0.6036 - val_loss: 0.9482 - val_acc: 0.6784
Epoch 7/10 7500/7500 [==============================] - 1516s - loss: 1.1600 - acc: 0.6087 - val_loss: 0.9682 - val_acc: 0.6797
Epoch 8/10 7500/7500 [==============================] - 1517s - loss: 1.1409 - acc: 0.6126 - val_loss: 0.9424 - val_acc: 0.6864
Epoch 9/10 7500/7500 [==============================] - 1521s - loss: 1.1387 - acc: 0.6142 - val_loss: 0.9183 - val_acc: 0.6915
Epoch 10/10 7500/7500 [==============================] - 1518s - loss: 1.1330 - acc: 0.6178 - val_loss: 0.9403 - val_acc: 0.6893 Begin Fine Tuning
Epoch 1/10 7500/7500 [==============================] - 2327s - loss: 1.0142 - acc: 0.6676 - val_loss: 0.6162 - val_acc: 0.7978
Epoch 2/10 7500/7500 [==============================] - 2333s - loss: 0.6800 - acc: 0.7705 - val_loss: 0.6005 - val_acc: 0.8130
Epoch 3/10 7500/7500 [==============================] - 2333s - loss: 0.5676 - acc: 0.8071 - val_loss: 0.5178 - val_acc: 0.8349
Epoch 4/10 7500/7500 [==============================] - 2333s - loss: 0.5044 - acc: 0.8279 - val_loss: 0.5559 - val_acc: 0.8302
Epoch 5/10 7500/7500 [==============================] - 2333s - loss: 0.4558 - acc: 0.8435 - val_loss: 0.4924 - val_acc: 0.8420
Epoch 6/10 7500/7500 [==============================] - 2333s - loss: 0.4254 - acc: 0.8535 - val_loss: 0.4852 - val_acc: 0.8448
Epoch 7/10 7500/7500 [==============================] - 2333s - loss: 0.3930 - acc: 0.8653 - val_loss: 0.5180 - val_acc: 0.8360
Epoch 8/10 7500/7500 [==============================] - 2333s - loss: 0.3661 - acc: 0.8741 - val_loss: 0.4862 - val_acc: 0.8535
Epoch 9/10 7500/7500 [==============================] - 2333s - loss: 0.3428 - acc: 0.8802 - val_loss: 0.4830 - val_acc: 0.8547
Epoch 10/10 7500/7500 [==============================] - 2334s - loss: 0.3217 - acc: 0.8877 - val_loss: 0.5089 - val_acc: 0.8519
